In [52]:
import os
import glob
from osgeo import gdal


In [53]:
year = 2022

In [62]:
# Multiband image Tif
dir_in_path = f'/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/{year}/'

# Output dir with the bands as vrts
out_dir = os.path.join(os.path.dirname(dir_in_path), f'bands_vrt_{year}')
os.makedirs(out_dir, exist_ok=True)

In [63]:
# out_dir

In [64]:
tile_paths = glob.glob(f"{dir_in_path}/*.tif")

In [65]:
base_name = f'GSE.V1.ANNUAL.{year}'

In [66]:
tile_paths

['/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/GSE_V1_ANNUAL_2022_1.tif',
 '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/GSE_V1_ANNUAL_2022_2.tif',
 '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/GSE_V1_ANNUAL_2022_3.tif',
 '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/GSE_V1_ANNUAL_2022_4.tif',
 '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/GSE_V1_ANNUAL_2022_5.tif',
 '/projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/GSE_V1_ANNUAL_2022_6.tif']

In [67]:
# Inspect first tile to discover band count and nodata
sample_ds = gdal.Open(tile_paths[0], gdal.GA_ReadOnly)
if sample_ds is None:
    raise RuntimeError(f'Could not open sample file: {tile_paths[0]}')

band_count = sample_ds.RasterCount
if band_count < 1:
    raise RuntimeError('Sample file reports zero bands.')

In [68]:
print(f'Found {len(tile_paths)} tiles, each with {band_count} bands:')
band_names = []
for i in range(1, band_count + 1):
    band = sample_ds.GetRasterBand(i)
    # Try to get a descriptive name
    desc = band.GetDescription()
    if not desc:
        desc = f'band_{i}'
    band_names.append(desc)
    # print(f'  Band {i}: {desc}')

Found 6 tiles, each with 64 bands:


In [69]:
band_count

64

In [70]:
# import rasterio
# img_path = tile_paths[0]

# with rasterio.open(img_path) as src:
#     print("NoData value:", src.nodata)
#     print("CRS:", src.crs)
#     print("Pixel size:", src.res)
#     print("Bounds:", src.bounds)

In [72]:
for b in range(1, band_count + 1):
    
    desc = band_names[b-1]
    out_vrt = os.path.join(out_dir, f'{base_name}.{desc}.vrt')

    # Use band-specific nodata if present on the sample; otherwise leave None

    opts = gdal.BuildVRTOptions(
        bandList=[b],
        separate=False,               # single-band output
        resampleAlg='nearest',        # change if you prefer 'bilinear' etc.
        resolution='highest',         # resolves mixed GSD (keeps native grid if consistent)
        allowProjectionDifference=False,  # safer: require same CRS
        srcNodata=-9999,               # force source nodata
        VRTNodata=-9999                # force VRT nodata
)

    # Build the VRT
    vrtds = gdal.BuildVRT(out_vrt, tile_paths, options=opts)
    if vrtds is None:
        raise RuntimeError(f'Failed to build VRT for band {b}')
    vrtds = None  # flush/close

    print(f'✓ {out_vrt}')

✓ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/bands_vrt_2022/GSE.V1.ANNUAL.2022.A00.vrt
✓ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/bands_vrt_2022/GSE.V1.ANNUAL.2022.A01.vrt
✓ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/bands_vrt_2022/GSE.V1.ANNUAL.2022.A02.vrt
✓ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/bands_vrt_2022/GSE.V1.ANNUAL.2022.A03.vrt
✓ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/bands_vrt_2022/GSE.V1.ANNUAL.2022.A04.vrt
✓ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/bands_vrt_2022/GSE.V1.ANNUAL.2022.A05.vrt
✓ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/bands_vrt_2022/GSE.V1.ANNUAL.2022.A06.vrt
✓ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbeddings/2022/bands_vrt_2022/GSE.V1.ANNUAL.2022.A07.vrt
✓ /projects/my-private-bucket/HLS-1DCNN-AGB/data/tif/GoogleEmbed